#   EQUIVALENCE TRIPS TO PATHS FOR CASE CS10.PP00.SO00

TOC:

[1. Download and format trips](#1.-download-and-format-trips)

[2. Download and format itineraries](#2.-download-and-format-itineraries)

[3. Assign trips to paths and analyse the results of the assignation](#3.-assign-trips-to-paths-and-analyse-the-results-of-the-assignation)

In [1]:
# libraries to import
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from ast import literal_eval
from typing import Dict
import os
print(os.getcwd())
os.chdir(r"C:\Users\LMENENDEZ\GitHub\MultiModX")
print(os.getcwd())
pd.set_option('display.max_columns', None)

c:\Users\LMENENDEZ\GitHub\MultiModX\notebooks
C:\Users\LMENENDEZ\GitHub\MultiModX


In [2]:
%load_ext autoreload

In [3]:
%autoreload
from strategic_evaluator.logit_model import *
from script.trips_format import *

## 1. Download and format trips

In [4]:
all_trips = pd.read_csv(
    r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP4 Performance Assessment Solution\Demand data\Matrices MITMA\with_archetypes\20220922_28_trip_matrix_arc_pt_processed.csv.gz",
    compression="gzip",
    sep="|"
)

In [5]:
#Here there is no trip id but later trip id appears all the time
trips = all_trips[all_trips["date"] == 20220923].reset_index(drop=True).rename(columns={"origin_nut": "origin", "destination_nut": "destination"})
trips.head()

,date,trip_period,origin_zone,origin,origin_name,destination_zone,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,distance,route_distance,duration,mode,service,legs,trip_vehicle_type,nationality,home_census,home_zone,overnight_census,income,age,sex,vehicle_type,short_professional_driver,trips,trips_km,sample_trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced
0,20220923,P00,01002,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,NF,O,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*01002*01002*None*train_13121*00-01*road*No...,other,ES,2_48,4801303,2_48,I01_[10000-15000),A01_[25-45),male,passenger,False,4.135,139.516,1.0,0.000000,0.000000,2.067500,2.067500,0.000000,0.000000,3,road-train-road,train_13121-train_13200,train_13121,train_13200,national,2,1,0,train_13121-train_13200,train_13106,train_13200
1,20220923,P00,01009_AM,ES211,Álava,31010_AM,ES220,Navarra,NaN,NaN,O,H,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01009_AM*01009_AM*None*train_11213*00-01*r...,other,ES,2_31,31010_AM,2_31,I01_[10000-15000),A01_[25-45),male,passenger,False,5.191,69.312,1.0,3.707857,0.000000,0.000000,0.494381,0.000000,0.988762,3,road-train-road,train_11213-train_11300,train_11213,train_11300,national,2,1,0,train_11213-train_11300,train_13106,train_11300
2,20220923,P00,01009_AM,ES211,Álava,abroad_208,abroad,abroad,NaN,ground_Fra_08,NF,NF,abroad,D05_[50000-inf),01-02,train,conv_unknown,P00*01009_AM*01009_AM*None*train_11213*00-01*r...,other,FR,NaN,NaN,NaN,NaN,NaN,NaN,passenger,False,1.599,309.588,1.0,0.888333,0.399750,0.142133,0.071067,0.062183,0.035533,3,road-train-road,train_11213-train_11600,train_11213,train_11600,international_D,2,1,0,train_11213-train_11600,train_11208,train_11511
3,20220923,P00,01036,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,H,O,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01036*01036*None*train_13106*00-01*road*No...,other,ES,2_01,01036,2_48,I02_[15000-inf),A02_[45-65),female,passenger,False,6.236,139.477,1.0,3.118000,0.000000,0.779500,1.559000,0.779500,0.000000,3,road-train-road,train_13106-train_13200,train_13106,train_13200,national,2,1,0,train_13106-train_13200,train_13106,train_13200
4,20220923,P00,0105902,ES211,Álava,09219,ES412,Burgos,NaN,NaN,O,NF,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*0105902*0105901*None*train_11208*00-01*roa...,other,ES,2_01,0105904,2_01,I02_[15000-inf),A02_[45-65),male,passenger,False,4.215,155.813,1.0,3.417568,0.113919,0.227838,0.227838,0.227838,0.000000,3,road-train-road,train_11208-train_11200,train_11208,train_11200,national,2,1,0,train_11208-train_11200,train_11208,train_11200


In [6]:
#associates each airport to the corresponding new NUTS
airports_to_NUTS={"airport_LPA":("ES705","Gran Canaria"),
                 "airport_FUE":("ES704","Fuerteventura"),
                 "airport_ACE":("ES708","Lanzarote"),
                 "airport_TFS":("ES709","Tenerife"),
                 "airport_TFN":("ES709","Tenerife"),
                 "airport_GMZ":("ES709","Tenerife"),
                 "airport_SPC":("ES707","La Palma"),
                 "airport_VDE":("ES703","El Hierro"),
                 "airport_PMI":("ES532","Mallorca"),
                 "airport_IBZ":("ES531","Eivissa i Formentera"),
                 "airport_MAH":("ES533","Menorca")}

In [7]:
trips=format_trips(trips, airports_to_NUTS)

17 columns were removed


We will remove:
- all the trips that come from or go to abroad
- the trips that go to Mad/Guadalajara because most likely these are cercanías
- all the trips containing weird stations

In [8]:
trips=trips[((trips["origin"].str.startswith("ES"))&(trips["destination"].str.startswith("ES")))]

In [9]:
trips=trips[~((trips["origin_zone"].str.strip().str.lower().str.startswith("abroad"))|(trips["destination_zone"].str.strip().str.lower().str.startswith("abroad")))]

In [10]:
trips=trips[~(((trips["origin"]=="ES424")&(trips["destination"]=="ES300"))|((trips["origin"]=="ES300")&(trips["destination"]=="ES424")))]

In [11]:
stops_loc=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\CS10\v=0.10\gtfs_es_UIC_v1.0\stops.txt").astype("str")
stops_loc["stop_id"] = stops_loc["stop_id"].apply(lambda x: "00" + x)

In [12]:
trips.loc[:,"weird_stations"] = trips["node_sequence_reduced"].apply(
    lambda x: find_weird_stations(x, stops_loc))

In [13]:
unique_weird_stations=get_weird_stations(trips["weird_stations"])

In [14]:
MobA_stations_coord=gpd.read_file(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain+abroad\v=0.1\datos moba\train_stations\train_stations.shp")

In [15]:
# identifies all the un-localisable stations
nowhere_stations=set(unique_weird_stations)-set(MobA_stations_coord["ID"])
print(f"there are {len(nowhere_stations)} stations that are not in the data provided by MobA but appear in the trips dataframe")

there are 199 stations that are not in the data provided by MobA but appear in the trips dataframe


In [16]:
trips = trips[~trips["node_sequence_reduced"].apply(lambda x: any(station in x for station in nowhere_stations))]

Now we have to construct a MMX path for each trip

In [17]:
# construct a dictionary with the equivalence between IATA and ICAO codes
airports=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\CS10\v=0.10\infrastructure\airports_info\IATA_ICAO_Airport_codes_v1.3.csv")
airports["ICAO"] = airports["ICAO"].fillna(airports["IATA"])
IATA_to_ICAO = airports.set_index("IATA")["ICAO"].to_dict()

In [18]:
# coordinates, geometry and other properties of all NUTS (in Europe?)
NUTS_coord=gpd.read_file(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP4 Performance Assessment Solution\Demand data\nuts3_2003_geom_10.gpkg")

In [19]:
# list of stations considered in MMX
train_stations_considered=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\CS10\v=0.10\infrastructure\rail_info\rail_stations_considered_GTFS_2022v0.1.csv").astype(str)
train_stations_considered["stop_id"] = train_stations_considered["stop_id"].apply(lambda x: "00" + x) #to make they start with 00

In [20]:
# align coordinate reference system (crs)
NUTS_coord = NUTS_coord.to_crs(MobA_stations_coord.crs)

# Perform a spatial join to find which NUTS region each station belongs to
spatial_join = gpd.sjoin(MobA_stations_coord, NUTS_coord, how="left", predicate="within")

# Construct the dictionary
station_to_NUTS = dict(zip(spatial_join["ID"], spatial_join["geocode"]))

In [21]:
train_stations_considered=format_train_stations_considered(train_stations_considered,station_to_NUTS)

In [22]:
# list of all MMX stations
train_station_MMX=train_stations_considered["stop_id"].tolist()

In [23]:
trips.head()

,date,origin_zone,origin,origin_name,destination_zone,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,legs,nationality,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp,trips,weird_stations
0,20220923,01001,ES211,Álava,20063,ES212,Guipúzcoa,NaN,NaN,H,NF,P06*01001*01001*None*train_11210*00-01*road*No...,ES,3.582000,0.0000,0.000000,0.000000,0.000000,0.000000,3,road-train-road,train_11210-train_11600,train_11210,train_11600,national,2,1,0,train_11210-train_11600,train_13106,train_11511,['rail'],3.582,[]
1,20220923,01001,ES211,Álava,28106,ES300,Madrid,NaN,NaN,NF,O,P14*01001*01051*None*train_11212*00-01*road*No...,ES,2.617719,0.0000,0.121754,0.213070,0.182632,0.334825,4,road-train-train-road,train_11212-train_11200-train_10000,train_11212,train_10000,national,2,2,0,train_11212-train_11200-train_10000,train_13106,train_18000,"['rail', 'rail']",3.470,[]
2,20220923,01001,ES211,Álava,31088,ES220,Navarra,NaN,NaN,H,NF,P20*01001*01001*None*train_11210*00-01*road*No...,ES,2.058750,0.2745,0.137250,0.686250,0.137250,0.000000,3,road-train-road,train_11210-train_80100,train_11210,train_80100,national,2,1,0,train_11210-train_80100,train_13106,train_80100,['rail'],3.294,[]
3,20220923,01001,ES211,Álava,31902,ES220,Navarra,NaN,NaN,O,W,P13*01001*01001*None*train_11210*00-01*road*No...,ES,3.064286,0.0000,0.000000,0.408571,0.000000,0.817143,3,road-train-road,train_11210-train_80100,train_11210,train_80100,national,2,1,0,train_11210-train_80100,train_11208,train_80100,['rail'],4.290,[]
4,20220923,01001,ES211,Álava,31908_AM,ES220,Navarra,NaN,NaN,NF,NF,P11*01001*01001*None*train_11210*00-01*road*No...,ES,3.767826,0.0000,0.000000,0.000000,0.565174,0.000000,3,road-train-road,train_11210-train_80005,train_11210,train_80005,national,2,1,0,train_11210-train_80005,train_11208,train_81202,['rail'],4.333,[]


In [24]:
trips=process_node_sequence_MMX(trips,train_station_MMX,IATA_to_ICAO)

## 2. Download and format itineraries

In [25]:
possible_itineraries=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\CS10\v=0.10\processed_cs10.pp00.so00_c1\paths_itineraries\possible_itineraries_0.csv")

In [26]:
possible_itineraries.columns

Index(['origin', 'destination', 'option', 'nservices', 'path',
       'total_travel_time', 'total_cost', 'total_emissions',
       'total_waiting_time', 'nmodes', 'journey_type', 'access_time',
       'egress_time', 'service_id_0', 'origin_0', 'destination_0',
       'provider_0', 'alliance_0', 'mode_0', 'departure_time_0',
       'arrival_time_0', 'travel_time_0', 'cost_0', 'service_cost_0',
       'emissions_cost_0', 'emissions_0', 'service_id_1', 'origin_1',
       'destination_1', 'provider_1', 'alliance_1', 'mode_1',
       'departure_time_1', 'arrival_time_1', 'travel_time_1', 'mct_time_0_1',
       'connecting_time_0_1', 'waiting_time_0_1', 'cost_1', 'service_cost_1',
       'emissions_cost_1', 'emissions_1'],
      dtype='object')

In [27]:
possible_itineraries=format_itineraries(possible_itineraries)

In [28]:
possible_itineraries

,origin,destination,path,nmodes,mode_tp,access_time,egress_time,travel_time_0,cost_0,emissions_0,travel_time_1,mct_time_0_1,cost_1,emissions_1,total_travel_time,total_cost,total_emissions
0,ES111,ES112,"['007131412', '007120300']",1,['rail'],43.0,74.0,179.500000,10.890000,3.410000,NaN,NaN,NaN,NaN,296.500000,10.890000,3.410000
1,ES111,ES112,"['007131412', '007122100', '007120300']",1,"['rail', 'rail']",43.0,74.0,67.882353,11.090000,3.470000,46.352941,10.0,3.360000,1.05000,241.235294,14.450000,4.520000
2,ES111,ES112,"['007131412', '007131400']",1,['rail'],43.0,101.0,29.000000,5.070000,1.590000,NaN,NaN,NaN,NaN,173.000000,5.070000,1.590000
3,ES111,ES113,"['007131400', '007122100']",1,['rail'],61.0,28.0,42.733333,7.400000,2.320000,NaN,NaN,NaN,NaN,131.733333,7.400000,2.320000
4,ES111,ES113,"['007131400', '007123004', '007122100']",1,"['rail', 'rail']",61.0,28.0,46.500000,4.670000,1.460000,74.500000,20.0,5.850000,1.83000,230.000000,10.520000,3.290000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26726,ES709,ES705,"['GCXO', 'GCRR', 'GCLP']",1,"['air', 'air']",120.0,57.0,50.000000,119.156500,51.910000,45.000000,45.0,108.466000,45.24000,317.000000,227.622500,97.150000
26727,ES709,ES708,"['GCTS', 'GCLP', 'GCRR']",1,"['air', 'air']",164.0,43.0,35.000000,89.585500,44.370000,45.000000,25.0,108.466000,45.24000,312.000000,198.051500,89.610000
26728,ES709,ES708,"['GCXO', 'GCLA', 'GCRR']",1,"['air', 'air']",120.0,43.0,30.000000,94.765500,44.370000,70.000000,20.0,138.296500,65.91000,283.000000,233.062000,110.280000
26729,ES709,ES708,"['GCXO', 'GCLP', 'GCRR']",1,"['air', 'air']",120.0,43.0,30.000000,88.149563,43.263750,44.687500,25.0,108.431687,45.01125,262.687500,196.581250,88.275000


## 3. Assign trips to paths and analyse the results of the assignation

In [29]:
trips=trips.rename({"node_sequence_MMX":"path"},axis=1)

In [30]:
trips.shape[0]

127998

In [31]:
trips_w_costs=trips.merge(possible_itineraries, on=["path","origin","destination"],how="left")

In [44]:
trips_w_costs=trips_w_costs[trips_w_costs["nmodes"].notna()]

In [45]:
trips_w_costs

,date,origin_zone,origin,origin_name,destination_zone,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,legs,nationality,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp_x,trips,weird_stations,path,nmodes,mode_tp_y,access_time,egress_time,travel_time_0,cost_0,emissions_0,travel_time_1,mct_time_0_1,cost_1,emissions_1,total_travel_time,total_cost,total_emissions
6,20220923,01002,ES211,Álava,0313903,ES521,Alicante / Alacant,NaN,NaN,H,NF,P07*01002*48904_AM*None*airport_BIO*01-02*road...,ES,3.565333,0.000000,0.000000,0.594222,0.594222,0.594222,3,road-plane-road,airport_BIO-airport_ALC,airport_BIO,airport_ALC,national,2,0,1,airport_BIO-airport_ALC,airport_BIO,airport_ALC,['air'],5.348,[],"['LEBB', 'LEAL']",1.0,['air'],146.0,72.0,80.0,153.862000,49.280000,NaN,NaN,NaN,NaN,298.0,153.862000,49.280000
7,20220923,01002,ES211,Álava,0801902,ES511,Barcelona,NaN,NaN,W,NF,P15*01002*0105901*None*train_11208*00-01*road*...,ES,3.330183,0.181646,0.605488,0.302744,0.363293,0.181646,3,road-train-road,train_11208-train_71801,train_11208,train_71801,national,2,1,0,train_11208-train_71801,train_11208,train_71801,['rail'],4.965,[],"['007111208', '007171801']",1.0,['rail'],24.0,36.0,309.0,45.270000,12.440000,NaN,NaN,NaN,NaN,369.0,45.270000,12.440000
8,20220923,01002,ES211,Álava,0905903,ES412,Burgos,NaN,NaN,NF,O,P13*01002*01036*None*train_13106*00-01*road*No...,ES,3.543867,0.000000,0.063283,0.000000,0.189850,0.000000,3,road-train-road,train_13106-train_11014,train_13106,train_11014,national,2,1,0,train_13106-train_11014,train_13106,train_11014,['rail'],3.797,[],"['007113106', '007111014']",1.0,['rail'],53.0,27.0,128.0,9.570000,3.000000,NaN,NaN,NaN,NaN,208.0,9.570000,3.000000
9,20220923,01002,ES211,Álava,0905909,ES412,Burgos,NaN,NaN,H,NF,P22*01002*01036*None*train_13106*00-01*road*No...,ES,4.125600,0.000000,0.305600,0.152800,0.000000,0.764000,3,road-train-road,train_13106-train_11014,train_13106,train_11014,national,2,1,0,train_13106-train_11014,train_13106,train_11014,['rail'],5.348,[],"['007113106', '007111014']",1.0,['rail'],53.0,27.0,128.0,9.570000,3.000000,NaN,NaN,NaN,NaN,208.0,9.570000,3.000000
190,20220923,01004_AM,ES211,Álava,07050,ES531,Eivissa i Formentera,NaN,NaN,H,NF,P17*01004_AM*48904_AM*None*airport_BIO*00-01*r...,ES,1.833500,0.000000,0.000000,0.916750,0.916750,0.000000,3,road-plane-road,airport_BIO-airport_IBZ,airport_BIO,airport_IBZ,national,2,0,1,airport_BIO-airport_IBZ,airport_BIO,airport_IBZ,['air'],3.667,[],"['LEBB', 'LEIB']",1.0,['air'],146.0,51.0,90.0,156.240000,56.000000,NaN,NaN,NaN,NaN,287.0,156.240000,56.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127524,20220923,PT300,ES709,Tenerife,3500402,ES708,Lanzarote,airport_TFN,NaN,NF,NF,P07*PT300*3802304*None*airport_TFN*00-01*plane...,ES,1.410500,1.989167,1.193500,0.054250,0.090417,0.036167,3,plane-plane-road,airport_TFN-airport_ACE,airport_TFN,airport_ACE,international_O,1,0,2,airport_TFN-airport_ACE,airport_TFN,airport_ACE,"['air', 'air']",4.774,[],"['GCXO', 'GCRR']",1.0,['air'],120.0,43.0,50.0,119.102962,51.553077,NaN,NaN,NaN,NaN,213.0,119.102962,51.553077
127526,20220923,PT300,ES709,Tenerife,3501601,ES705,Gran Canaria,airport_TFN,NaN,NF,NF,P06*PT300*3802304*None*airport_TFN*00-01*plane...,ES,0.738776,1.080459,0.738776,0.036939,0.083112,0.036939,3,plane-plane-road,airport_TFN-airport_LPA,airport_TFN,airport_LPA,international_O,1,0,2,airport_TFN-airport_LPA,airport_TFN,airport_LPA,"['air', 'air']",2.715,[],"['GCXO', 'GCLP']",1.0,['air'],120.0,57.0,30.0,88.205638,43.637586,NaN,NaN,NaN,NaN,207.0,88.205638,43.637586
127527,20220923,PT300,ES709,Tenerife,3501802,ES708,Lanzarote,airport_TFN,NaN,NF,H,P06*PT300*3802304*N

In [46]:
num_trips=trips["trips"].sum()
num_assigned=trips_w_costs[trips_w_costs["path"].notna()]["trips"].sum()
print(f"we assigned {num_assigned/num_trips*100:.2f}% of the trips")

we assigned 47.21% of the trips


In [47]:
trips_logit =trips_w_costs.groupby(['origin', 'destination', 'path']).agg({  
    'trips': 'sum',    # Sum num_of_trips
    'archetype_0': 'sum',     # Sum archetype_0
    'archetype_1': 'sum',     # Sum archetype_1
    'archetype_2': 'sum',     # Sum archetype_2
    'archetype_3': 'sum',     # Sum archetype_3
    'archetype_4': 'sum',     # Sum archetype_4
    'archetype_5': 'sum',     # Sum archetype_5
    'nmodes': 'first',        # Keep the first value for nmodes
    'access_time': 'first',   # Keep the first value for access_time
    'egress_time': 'first',    # Keep the first value for egress_time
    'travel_time_0': 'first',
    'cost_0': 'first',
    'emissions_0':'first',
    'mct_time_0_1': 'first',
    'travel_time_1': 'first',
    'cost_1': 'first',
    'emissions_1':'first',
    #'mct_time_1_2': 'first',
    #'travel_time_2': 'first',
    #'cost_2': 'first',
    #'emissions_2':'first',
    'total_travel_time': 'first',
    'total_cost':'first',
    'total_emissions':'first'
}).reset_index()

Note that when doing this we are removing all the trips that have path==NaN

In [48]:
trips_logit["trips"].sum()

np.float64(240055.254)

In [49]:
trips_w_costs[trips_w_costs["path"].notna()]["trips"].sum()

np.float64(240055.25400000002)

In [37]:
trips["trips"].sum()

np.float64(508441.81299999997)

In [51]:
trips_logit.to_csv("trips_logit_CS10.csv",index=False)

## 2. BIS download and format itineraries with 2 connections

In [25]:
possible_itineraries_2_conn=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\CS10\v=0.10\processed_cs10.pp00.so00_2conn\paths_itineraries\possible_itineraries_0.csv")

In [26]:
possible_itineraries_2_conn=format_itineraries(possible_itineraries_2_conn)

## 3. BIS Assign trips to paths and analyse the results of the assignation

In [27]:
trips=trips.rename({"node_sequence_MMX":"path"},axis=1)

In [28]:
trips.shape[0]

127998

In [29]:
trips_w_costs=trips.merge(possible_itineraries_2_conn, on=["path","origin","destination"],how="left")

In [30]:
trips_w_costs.shape[0]

127998

In [31]:
trips_w_costs=trips_w_costs[trips_w_costs["nmodes"].notna()]

In [32]:
num_trips=trips["trips"].sum()
num_assigned=trips_w_costs[trips_w_costs["path"].notna()]["trips"].sum()
print(f"we assigned {num_assigned/num_trips*100:.2f}% of the trips")

we assigned 47.21% of the trips


In [33]:
trips_logit =trips_w_costs.groupby(['origin', 'destination', 'path']).agg({  
    'trips': 'sum',    # Sum num_of_trips
    'archetype_0': 'sum',     # Sum archetype_0
    'archetype_1': 'sum',     # Sum archetype_1
    'archetype_2': 'sum',     # Sum archetype_2
    'archetype_3': 'sum',     # Sum archetype_3
    'archetype_4': 'sum',     # Sum archetype_4
    'archetype_5': 'sum',     # Sum archetype_5
    'nmodes': 'first',        # Keep the first value for nmodes
    'access_time': 'first',   # Keep the first value for access_time
    'egress_time': 'first',    # Keep the first value for egress_time
    'travel_time_0': 'first',
    'cost_0': 'first',
    'emissions_0':'first',
    'mct_time_0_1': 'first',
    'travel_time_1': 'first',
    'cost_1': 'first',
    'emissions_1':'first',
    'mct_time_1_2': 'first',
    'travel_time_2': 'first',
    'cost_2': 'first',
    'emissions_2':'first',
    'total_travel_time': 'first',
    'total_cost':'first',
    'total_emissions':'first'
}).reset_index()

In [34]:
trips_logit.to_csv("trips_logit_CS10_2_conn.csv",index=False)

In [35]:
trips_logit[trips_logit["path"].notna()].sample(10)

,origin,destination,path,trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,nmodes,access_time,egress_time,travel_time_0,cost_0,emissions_0,mct_time_0_1,travel_time_1,cost_1,emissions_1,mct_time_1_2,travel_time_2,cost_2,emissions_2,total_travel_time,total_cost,total_emissions
418,ES213,ES618,"['LEBB', 'LEZL']",286.240,206.580084,4.022728,7.820028,31.184479,25.370312,11.262369,1.0,101.0,44.0,82.500000,155.29,59.435000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,227.500000,155.29,59.435000
2579,ES709,ES300,"['GCTS', 'LEMD']",400.958,241.383492,11.929664,9.390994,76.277307,34.488485,27.488058,1.0,164.0,63.0,168.333333,216.89,111.456667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,395.333333,216.89,111.456667
951,ES416,ES532,"['LEMD', 'LEPA']",4.563,2.737800,0.547560,0.000000,0.547560,0.365040,0.365040,1.0,202.0,79.0,85.000000,142.40,48.228750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,366.000000,142.40,48.228750
1190,ES425,ES421,"['007160300', '007160600']",7.999,6.833311,0.049798,0.025392,0.500996,0.279760,0.309743,1.0,98.0,20.0,84.000000,13.44,4.210000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202.000000,13.44,4.210000
611,ES300,ES111,"['007117000', '007131412']",305.522,211.458661,8.685132,9.337820,32.919605,18.809294,24.311489,1.0,32.0,36.0,230.000000,53.35,14.660000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,298.000000,53.35,14.660000
679,ES300,ES421,"['007117000', '007160600']",212.793,183.698443,1.160338,2.795829,8.298650,8.264516,8.575224,1.0,32.0,20.0,108.214286,21.00,6.580000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160.214286,21.00,6.580000
936,ES416,ES412,"['007108004', '007111014']",13.402,11.628106,0.086127,0.000000,1.214072,0.258380,0.215316,1.0,41.0,27.0,84.666667,15.40,4.820000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152.666667,15.40,4.820000
1756,ES523,ES421,"['007165000', '007160600']",340.477,263.009151,7.659788,5.389774,26.877883,19.429703,18.110700,1.0,33.0,20.0,131.666667,12.68,3.970000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,184.666667,12.68,3.970000
251,ES130,ES521,"['LEXJ', 'LEMD', 'LEAL']",5.942,1.980667,0.000000,0.000000,0.990333,0.000000,2.971000,1.0,106.0,72.0,66.666667,118.81,47.910000,47.0,65.0,122.77,55.04,NaN,NaN,NaN,NaN,356.666667,241.58,102.950000
968,ES418,ES114,"['007108240', '007122100']",6.867,4.245708,0.131827,0.205186,1.156302,0.366429,0.761548,1.0,54.0,91.0,123.666667,25.13,7.870000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,268.666667,25.13,7.870000


In [36]:
trips_logit[trips_logit["nmodes"].isna()]

,origin,destination,path,trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,nmodes,access_time,egress_time,travel_time_0,cost_0,emissions_0,mct_time_0_1,travel_time_1,cost_1,emissions_1,mct_time_1_2,travel_time_2,cost_2,emissions_2,total_travel_time,total_cost,total_emissions
